# Group : SM_1

Group Members :
Sushruti Acharya , Ashutosh Sharma, Adesh Valecha

## Data : UTA Accident Data

The data has been provided by UTA department.

We are using the UTA accident data of last Six years (2013-2018).
The are two different datasets we are using:

1. Camera Flag Data Set
2. Accident Data Set

# Data mining task - Classification

### Building classification model to classify the accident event was avoidable or not.

## Data Preparation & Exploration  

In [2]:
# Import RFormula, classifiers and MulticlassClassificationEvaluator packages 
# necessary for building and evaluating baseline classification and regression models

from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression\
,LinearSVC, MultilayerPerceptronClassifier,NaiveBayes\
,GBTClassifier,RandomForestClassifier, DecisionTreeClassifier

from pyspark.ml.feature import RFormula, StringIndexer, VectorAssembler, VectorIndexer, OneHotEncoder
from pyspark.ml import Pipeline
# Model building
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
# Model evaluation
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RegressionMetrics
# Parameter tuning
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
# Feature engineering
from pyspark.ml.feature import Normalizer, ChiSqSelector

In [3]:
# Import the functions in the pyspark.sql package
from pyspark.sql.functions import col, desc, asc, round, corr, countDistinct, min, max, sum, count, avg, expr, to_date, year

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [5]:
#Reading UTA Accident Data

data = spark.read.csv("E:/Summer_Sem/Big_Data/project/UTA_accidents.csv", header = True, inferSchema="true")
data.printSchema()

root
 |-- Incident Address City: string (nullable = true)
 |-- Incident Date: string (nullable = true)
 |-- Incident Description: string (nullable = true)
 |-- ARC Status: string (nullable = true)
 |-- ARC Decision: string (nullable = true)
 |-- Incident Weather Condition: string (nullable = true)
 |-- Incident Road Surface Condition: string (nullable = true)
 |-- Incident Light Condition: string (nullable = true)
 |-- UTA Vehicle Behavior: string (nullable = true)
 |-- Incident Location Type: string (nullable = true)
 |-- Incident Categorical Description: string (nullable = true)
 |-- UTA Vehicle Number: string (nullable = true)
 |-- Vehicle One Behavior: string (nullable = true)
 |-- Vehicle Two Behavior: string (nullable = true)
 |-- Vehicle Three Behavior: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- yearmonth: integer (nullable = true)
 |-- addressid (OpAnalytics): integer (nullable = true)
 |-- Incident Time of Day (Seco

In [6]:
data = data.drop('Incident Categorical Description','Incident Description','UTA Vehicle Number','addressid (OpAnalytics)')

In [7]:
data_df = data.where(col("ARC Status")=="ARCR")

In [8]:
data_df = data_df.where(col("ARC Decision")!="UNKNOWN")

In [9]:
data_df = data_df.where(col("ARC Decision")!="TIE")

In [10]:
data_df.groupby('ARC Decision').count().show()

+------------+-----+
|ARC Decision|count|
+------------+-----+
|        CLS1|  672|
|        CLS0|   83|
|        CLS4|    4|
|          UN| 2241|
|        CLS2|  136|
|        CLS3|   22|
+------------+-----+



In [11]:
from pyspark.sql.functions import when
data_df2 = data_df.withColumn("ACC_STATUS", when(col('ARC Decision')=="UN",0).otherwise(1))
data_df2.select('ARC Decision','ACC_STATUS').show(15)

+------------+----------+
|ARC Decision|ACC_STATUS|
+------------+----------+
|        CLS1|         1|
|          UN|         0|
|        CLS1|         1|
|          UN|         0|
|          UN|         0|
|        CLS1|         1|
|        CLS1|         1|
|        CLS1|         1|
|        CLS2|         1|
|          UN|         0|
|          UN|         0|
|          UN|         0|
|          UN|         0|
|          UN|         0|
|        CLS1|         1|
+------------+----------+
only showing top 15 rows



In [12]:
data_df2.printSchema()

root
 |-- Incident Address City: string (nullable = true)
 |-- Incident Date: string (nullable = true)
 |-- ARC Status: string (nullable = true)
 |-- ARC Decision: string (nullable = true)
 |-- Incident Weather Condition: string (nullable = true)
 |-- Incident Road Surface Condition: string (nullable = true)
 |-- Incident Light Condition: string (nullable = true)
 |-- UTA Vehicle Behavior: string (nullable = true)
 |-- Incident Location Type: string (nullable = true)
 |-- Vehicle One Behavior: string (nullable = true)
 |-- Vehicle Two Behavior: string (nullable = true)
 |-- Vehicle Three Behavior: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- yearmonth: integer (nullable = true)
 |-- Incident Time of Day (Seconds): integer (nullable = true)
 |-- StopId (Trapeze): integer (nullable = true)
 |-- tripid: integer (nullable = true)
 |-- lineid: integer (nullable = true)
 |-- linedirectionid: integer (nullable = true)
 |-- buid: inte

In [13]:
data_df2.select(countDistinct("ACC_STATUS")).show()

+--------------------------+
|count(DISTINCT ACC_STATUS)|
+--------------------------+
|                         2|
+--------------------------+



In [14]:
data_df2.groupBy('ACC_STATUS').count().show()

+----------+-----+
|ACC_STATUS|count|
+----------+-----+
|         1|  917|
|         0| 2241|
+----------+-----+



## Cross - tabulation

We are checking the frequency distribution of set of variables with respect to accident status.

In [77]:
data_df2.stat.crosstab("Incident Weather Condition","ACC_STATUS").show()
data_df2.stat.crosstab("Incident Road Surface Condition","ACC_STATUS").show()
data_df2.stat.crosstab("Incident Light Condition","ACC_STATUS").show()
data_df2.stat.crosstab("UTA Vehicle Behavior","ACC_STATUS").show()
data_df2.stat.crosstab("Incident Location Type","ACC_STATUS").show()
data_df2.stat.crosstab("Fuel Type","ACC_STATUS").show()
data_df2.stat.crosstab("Vehicle One Behavior","ACC_STATUS").show()
data_df2.stat.crosstab("Vehicle Two Behavior","ACC_STATUS").show()

+-------------------------------------+----+---+
|Incident Weather Condition_ACC_STATUS|   0|  1|
+-------------------------------------+----+---+
|                                  N/A|   8|  5|
|                        PARTLY CLOUDY|   8|  0|
|                                 SNOW|  11|  2|
|                           LIGHT RAIN|  71| 30|
|                           HEAVY SNOW|  54| 13|
|                            LIGHT FOG|   4|  4|
|                               CLOUDY| 385|131|
|                           HEAVY RAIN|  14|  6|
|                           LIGHT SNOW|  64| 14|
|                                 RAIN|   4|  5|
|                            HEAVY FOG|   2|  0|
|                                CLEAR|1595|700|
|                                  FOG|   1|  0|
|                        UNCATEGORIZED|   1|  1|
|                              UNKNOWN|  19|  6|
+-------------------------------------+----+---+

+------------------------------------------+----+---+
|Incident Road

## Schema of the prepared data

1. We have considered only those records which are reviewed by the accident review committee, where ARC Status = "ARCR".
2. Removed columns such as Incident Categorical Description','Incident Description','UTA Vehicle Number','addressid (OpAnalytics) as they are not significant features for data mining tasks. 
3. Additionally, considered only those records with ARC Decision as avoidable and unaviodable. 
4. Removed records with NULL values.
5. Renamed the columns as RFormula accept column names without spaces.


In [15]:
#subset
sub_df = data_df2.select("Incident Weather Condition","Incident Road Surface Condition","Incident Light Condition","UTA Vehicle Behavior","Incident Location Type","Fuel Type","Vehicle One Behavior","Vehicle Two Behavior","ACC_STATUS")

In [16]:
sub_df.printSchema()

root
 |-- Incident Weather Condition: string (nullable = true)
 |-- Incident Road Surface Condition: string (nullable = true)
 |-- Incident Light Condition: string (nullable = true)
 |-- UTA Vehicle Behavior: string (nullable = true)
 |-- Incident Location Type: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Vehicle One Behavior: string (nullable = true)
 |-- Vehicle Two Behavior: string (nullable = true)
 |-- ACC_STATUS: integer (nullable = false)



In [17]:
sub_df = sub_df.withColumnRenamed("Incident Weather Condition","Incident_Weather_Condition")
sub_df = sub_df.withColumnRenamed("Incident Road Surface Condition","Incident_Road_Surface_Condition")
sub_df = sub_df.withColumnRenamed("UTA Vehicle Behavior","UTA_Vehicle_Behavior")
sub_df = sub_df.withColumnRenamed("Incident Location Type","Incident_Location_Type")
sub_df = sub_df.withColumnRenamed("Incident Light Condition","Incident_Light_Condition")
sub_df = sub_df.withColumnRenamed("Incident Time of Day (Seconds)","Incident_Time")
sub_df = sub_df.withColumnRenamed("Fuel Type","Fuel_Type")
sub_df = sub_df.withColumnRenamed("Vehicle One Behavior","Vehicle_One_Behavior")
sub_df = sub_df.withColumnRenamed("Vehicle Two Behavior","Vehicle_Two_Behavior")
sub_df.printSchema()

root
 |-- Incident_Weather_Condition: string (nullable = true)
 |-- Incident_Road_Surface_Condition: string (nullable = true)
 |-- Incident_Light_Condition: string (nullable = true)
 |-- UTA_Vehicle_Behavior: string (nullable = true)
 |-- Incident_Location_Type: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Vehicle_One_Behavior: string (nullable = true)
 |-- Vehicle_Two_Behavior: string (nullable = true)
 |-- ACC_STATUS: integer (nullable = false)



In [18]:
sub_df.groupBy('Incident_Weather_Condition').count().show()
sub_df.groupBy('Incident_Road_Surface_Condition').count().show()
sub_df.groupBy('Incident_Light_Condition').count().show()
sub_df.groupBy('ACC_STATUS').count().show()

+--------------------------+-----+
|Incident_Weather_Condition|count|
+--------------------------+-----+
|             UNCATEGORIZED|    2|
|                 LIGHT FOG|    8|
|                 HEAVY FOG|    2|
|                    CLOUDY|  516|
|                LIGHT RAIN|  101|
|                HEAVY RAIN|   20|
|                   UNKNOWN|   25|
|                     CLEAR| 2295|
|             PARTLY CLOUDY|    8|
|                      SNOW|   13|
|                       N/A|   13|
|                LIGHT SNOW|   78|
|                      RAIN|    9|
|                       FOG|    1|
|                HEAVY SNOW|   67|
+--------------------------+-----+

+-------------------------------+-----+
|Incident_Road_Surface_Condition|count|
+-------------------------------+-----+
|                  UNCATEGORIZED|    6|
|                            DRY| 2628|
|                            WET|  274|
|                        UNKNOWN|   25|
|                          MUDDY|    3|
|             

In [19]:
sub_df = sub_df.na.drop()
sub_df.groupBy('Incident_Weather_Condition').count().show()

+--------------------------+-----+
|Incident_Weather_Condition|count|
+--------------------------+-----+
|             UNCATEGORIZED|    1|
|                 LIGHT FOG|    7|
|                 HEAVY FOG|    2|
|                    CLOUDY|  414|
|                LIGHT RAIN|   73|
|                HEAVY RAIN|   17|
|                   UNKNOWN|   13|
|                     CLEAR| 1843|
|             PARTLY CLOUDY|    8|
|                      SNOW|   11|
|                       N/A|    8|
|                LIGHT SNOW|   69|
|                      RAIN|    9|
|                HEAVY SNOW|   56|
+--------------------------+-----+



In [20]:
sub_df.groupBy('ACC_STATUS').count().show()

+----------+-----+
|ACC_STATUS|count|
+----------+-----+
|         1|  703|
|         0| 1828|
+----------+-----+



## Applied RFormula for data preparation

In [50]:
formula1 = RFormula(formula="ACC_STATUS ~ .", featuresCol="features", labelCol="label")

fitted_rf1 = formula1.fit(sub_df)

CD1_prepared = fitted_rf1.transform(sub_df)
CD1_prepared.printSchema()
CD1_prepared.show(5)

root
 |-- Incident_Weather_Condition: string (nullable = true)
 |-- Incident_Road_Surface_Condition: string (nullable = true)
 |-- Incident_Light_Condition: string (nullable = true)
 |-- UTA_Vehicle_Behavior: string (nullable = true)
 |-- Incident_Location_Type: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Vehicle_One_Behavior: string (nullable = true)
 |-- Vehicle_Two_Behavior: string (nullable = true)
 |-- ACC_STATUS: integer (nullable = false)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)

+--------------------------+-------------------------------+------------------------+--------------------+----------------------+--------------------+--------------------+--------------------+----------+--------------------+-----+
|Incident_Weather_Condition|Incident_Road_Surface_Condition|Incident_Light_Condition|UTA_Vehicle_Behavior|Incident_Location_Type|           Fuel_Type|Vehicle_One_Behavior|Vehicle_Two_Behavior|ACC_STATUS|          

## Used ChiSqSelector to select the top 7 features for classification task


In [53]:
from pyspark.ml.feature import ChiSqSelector

chiSqrModel = ChiSqSelector(numTopFeatures=7, featuresCol= "features", outputCol="selectedFeatures", labelCol="label")
selectedData = chiSqrModel.fit(CD1_prepared).transform(CD1_prepared)

selectedData.show(5)

chisq_train, chisq_test = selectedData.select('label','selectedFeatures').randomSplit([0.7,0.3], seed = 100)

+--------------------------+-------------------------------+------------------------+--------------------+----------------------+--------------------+--------------------+--------------------+----------+--------------------+-----+--------------------+
|Incident_Weather_Condition|Incident_Road_Surface_Condition|Incident_Light_Condition|UTA_Vehicle_Behavior|Incident_Location_Type|           Fuel_Type|Vehicle_One_Behavior|Vehicle_Two_Behavior|ACC_STATUS|            features|label|    selectedFeatures|
+--------------------------+-------------------------------+------------------------+--------------------+----------------------+--------------------+--------------------+--------------------+----------+--------------------+-----+--------------------+
|                    CLOUDY|                            DRY|                DARKNESS|      GOING STRAIGHT|               UNKNOWN|D  -Diesel Fuel  ...|      Going Straight|                 N/A|         1|(135,[1,13,21,26,...|  1.0|       (7,[5],

## Normalization of the data

In [54]:
# Normalize each Vector using L1 norm.
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
normData = normalizer.transform(CD1_prepared)
normData.printSchema()
norm_train, norm_test = normData.randomSplit([0.7, 0.3], seed=2018)

root
 |-- Incident_Weather_Condition: string (nullable = true)
 |-- Incident_Road_Surface_Condition: string (nullable = true)
 |-- Incident_Light_Condition: string (nullable = true)
 |-- UTA_Vehicle_Behavior: string (nullable = true)
 |-- Incident_Location_Type: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Vehicle_One_Behavior: string (nullable = true)
 |-- Vehicle_Two_Behavior: string (nullable = true)
 |-- ACC_STATUS: integer (nullable = false)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- normFeatures: vector (nullable = true)



## Model building and Evaluation


In [55]:
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy as np

def classifier_evaluator (predictions):
    predictionAndLabels =predictions.select("prediction", "label").rdd
    metrics = MulticlassMetrics(predictionAndLabels)
    print("Precision(class1):", np.round(metrics.precision(0.0),3))
    print("Recall(class1):", np.round(metrics.recall(0.0),3))
    print("F-Score(class1):", np.round(metrics.fMeasure(0.0, beta=1.0),3))
    print("Precision(class2):", np.round(metrics.precision(1.0),3))
    print("Recall(class2):", np.round(metrics.recall(1.0),3))
    print("F-Score(class2):", np.round(metrics.fMeasure(1.0, beta=1.0),3))
    print("Accuracy:", np.round(metrics.accuracy,3))   
    

## 1. Decision Tree Classifier

In [56]:
#Decision Tree Classifier using normalized data

dt_norm = DecisionTreeClassifier(labelCol='label',featuresCol='normFeatures')
CD1_fittedDT = dt_norm.fit(norm_train)

CD1_dt_test_results = CD1_fittedDT.transform(norm_test)
classifier_evaluator(CD1_dt_test_results)



Precision(class1): 0.863
Recall(class1): 0.858
F-Score(class1): 0.86
Precision(class2): 0.639
Recall(class2): 0.649
F-Score(class2): 0.644
Accuracy: 0.799


In [57]:
#Decision Tree Classifier - selecting top 5 features using Chi-square

dt_chisq = DecisionTreeClassifier(labelCol='label',featuresCol='selectedFeatures')
CD1_fittedDT = dt_chisq.fit(chisq_train)

CD1_dt_test_results = CD1_fittedDT.transform(chisq_test)
classifier_evaluator(CD1_dt_test_results)

Precision(class1): 0.781
Recall(class1): 0.917
F-Score(class1): 0.843
Precision(class2): 0.57
Recall(class2): 0.299
F-Score(class2): 0.392
Accuracy: 0.751


We are evaluating decision tree classifier model on Normalized data and using top7 features provided by Chisquare method.
We observed that we are getting better accuracy results on using normalized data.

Hence, we are using normalized data for building different classification models.

## 2. Random Forest Classifier

In [59]:
#Random Forest Classifier using normalized data

rfc_norm = RandomForestClassifier(labelCol='label',featuresCol='normFeatures')
CD1_fitted_rfc = rfc_norm.fit(norm_train)

CD1_rfc_test_results = CD1_fitted_rfc.transform(norm_test)
classifier_evaluator(CD1_rfc_test_results)

Precision(class1): 0.756
Recall(class1): 0.969
F-Score(class1): 0.849
Precision(class2): 0.709
Recall(class2): 0.193
F-Score(class2): 0.304
Accuracy: 0.752


## 3. Gradient Boost Tree Classifier

In [60]:
#Gradient Boost Tree Classifier using normalized data

gbt_norm = GBTClassifier(labelCol='label',featuresCol='normFeatures')
CD1_fittedGBT = gbt_norm.fit(norm_train)

CD1_gbt_test_results = CD1_fittedGBT.transform(norm_test)
classifier_evaluator(CD1_gbt_test_results)

Precision(class1): 0.851
Recall(class1): 0.864
F-Score(class1): 0.857
Precision(class2): 0.634
Recall(class2): 0.609
F-Score(class2): 0.621
Accuracy: 0.793


## 4. SVN Classifier

In [61]:
#SVN Classifier using normalized data

svm_norm = LinearSVC(labelCol='label',featuresCol='normFeatures')
CD1_fittedSVM = svm_norm.fit(norm_train)

CD1_svm_test_results = CD1_fittedSVM.transform(norm_test)
classifier_evaluator(CD1_svm_test_results)

Precision(class1): 0.848
Recall(class1): 0.879
F-Score(class1): 0.863
Precision(class2): 0.656
Recall(class2): 0.594
F-Score(class2): 0.623
Accuracy: 0.799


On comparing the evaluation metric results we found that Decision Tree Classifier and SVN are performing better than the other classification models.

# Data mining task - Regression

## Building Regression model to predict the score of the driver based upon different variables from the data.

In [128]:
# Load orders in a csv file into a Spark DataFrame. 
# Open this csv file via the file directory in your Jupyter Notebook before running this cell.
df = spark.read.csv("Camera_flag.csv", header = True, inferSchema="true")

In [129]:
# Load orders in a csv file into a Spark DataFrame. 
# Open this csv file via the file directory in your Jupyter Notebook before running this cell.
df2 = spark.read.csv("Accident_Data.csv", header = True, inferSchema="true")

## Joining Accident and Camera Flag data sets

In [130]:
# Join expression
joinExpression = df2["Driver ID"] == df["Driver ID"]
# Join type
joinType = "left"
# Join command
left_join_df = df.join(df2, joinExpression, joinType)
# Drop redundant columns
#left_join_df = left_join_df.drop(df2["UTA Vehicle Number"]).drop(df2["Market"])
left_join_df = left_join_df.drop('Driver ID')
left_join_df.printSchema()
left_join_df.show(5)
# Explore data
#left_join_df.select("Returned").distinct().show()
#left_join_df.select(count("Order ID")).show()

root
 |-- Coaching: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Event ID: string (nullable = true)
 |-- Recorded Date: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Observations: string (nullable = true)
 |-- Vehicle: string (nullable = true)
 |-- Incident Address City: string (nullable = true)
 |-- Incident Date: string (nullable = true)
 |-- Incident Description: string (nullable = true)
 |-- ARC Status: string (nullable = true)
 |-- ARC Decision: string (nullable = true)
 |-- Incident Weather Condition: string (nullable = true)
 |-- Incident Road Surface Condition: string (nullable = true)
 |-- Incident Light Condition: string (nullable = true)
 |-- UTA Vehicle Behavior: string (nullable = true)
 |-- Incident Location Type: string (nullable = true)
 |-- Incident Categorical Description: string (nullable = true)
 |-- UTA Vehicle Number: string (nullable = true)
 |-- Vehicle One Behavior: string (nulla

## Schema of the prepared data

1. We have considered only those records which are reviewed by the accident review committee, where ARC Status = "ARCR".
2. Removed columns such as Incident Categorical Description','Incident Description','UTA Vehicle Number','addressid (OpAnalytics) as they are not significant features for data mining tasks.  
3. Removed records with NULL values.
4. Renamed the columns as RFormula accept column names without spaces.

In [131]:
df_reg = left_join_df.drop('StopId (Trapeze)','Vehicle','Recorded Date','Event ID','Flag','Coaching','Incident Address City','Incident Date','Incident Description','Incident Categorical Description','Vehicle Three Behavior','year','month','yearmonth','addressid (OpAnalytics)','StopId ','tripid','lineid','linedirectionid','buid','divisionid','modeid','serviceid','FromTime','toTime','vehicleid','Operator Gender','Operator Job Description','Equipment Class','Vehicle Length (ft)','Vehicle Width','fleetCode','Manufacturer','Make & Model','Model Year','Fuel Type')


df_reg.printSchema()

root
 |-- Severity: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- Observations: string (nullable = true)
 |-- ARC Status: string (nullable = true)
 |-- ARC Decision: string (nullable = true)
 |-- Incident Weather Condition: string (nullable = true)
 |-- Incident Road Surface Condition: string (nullable = true)
 |-- Incident Light Condition: string (nullable = true)
 |-- UTA Vehicle Behavior: string (nullable = true)
 |-- Incident Location Type: string (nullable = true)
 |-- UTA Vehicle Number: string (nullable = true)
 |-- Vehicle One Behavior: string (nullable = true)
 |-- Vehicle Two Behavior: string (nullable = true)
 |-- Incident Time of Day (Seconds): integer (nullable = true)



In [66]:
df_reg = df_reg.where(col("ARC Status")=="ARCR")
df_reg = df_reg.where(col("ARC Decision")!="UNKNOWN")
df_reg = df_reg.where(col("ARC Decision")!="TIE")

In [67]:
df_reg = df_reg.na.drop()

In [68]:
df_reg = df_reg.withColumnRenamed("ARC Status","ARC_Status")
df_reg = df_reg.withColumnRenamed("ARC Decision","ARC_Decision")
df_reg = df_reg.withColumnRenamed("Incident Weather Condition","Incident_Weather_Condition")
df_reg = df_reg.withColumnRenamed("Incident Road Surface Condition","Incident_Road_Surface_Condition")
df_reg = df_reg.withColumnRenamed("UTA Vehicle Behavior","UTA_Vehicle_Behavior")
df_reg = df_reg.withColumnRenamed("Incident Location Type","Incident_Location_Type")
df_reg = df_reg.withColumnRenamed("Incident Light Condition","Incident_Light_Condition")
df_reg = df_reg.withColumnRenamed("Incident Time of Day (Seconds)","Incident_Time")
df_reg = df_reg.withColumnRenamed("Vehicle One Behavior","Vehicle_One_Behavior")
df_reg = df_reg.withColumnRenamed("Vehicle Two Behavior","Vehicle_Two_Behavior")

In [69]:
df_sub_rg = df_reg.select("Score","Incident_Weather_Condition","Incident_Road_Surface_Condition","UTA_Vehicle_Behavior","Incident_Location_Type","Incident_Light_Condition","Incident_Time","Vehicle_One_Behavior","Vehicle_Two_Behavior","Observations","Severity","ARC_Decision")

In [70]:
df_sub_rg.printSchema()

root
 |-- Score: integer (nullable = true)
 |-- Incident_Weather_Condition: string (nullable = true)
 |-- Incident_Road_Surface_Condition: string (nullable = true)
 |-- UTA_Vehicle_Behavior: string (nullable = true)
 |-- Incident_Location_Type: string (nullable = true)
 |-- Incident_Light_Condition: string (nullable = true)
 |-- Incident_Time: integer (nullable = true)
 |-- Vehicle_One_Behavior: string (nullable = true)
 |-- Vehicle_Two_Behavior: string (nullable = true)
 |-- Observations: string (nullable = true)
 |-- Severity: string (nullable = true)
 |-- ARC_Decision: string (nullable = true)



In [79]:
# Statistical summary of numeric data - 'Score'

df_sub_rg.describe('Score').show()

+-------+-----------------+
|summary|            Score|
+-------+-----------------+
|  count|            67503|
|   mean|159.3882938536065|
| stddev|86.57667861387961|
|    min|                0|
|    max|              610|
+-------+-----------------+



## Applied RFormula for preparing data

In [104]:
formula = RFormula(
    formula="Score ~ .",
    featuresCol="features",
    labelCol="label")

df_rf_fitted = formula.fit(df_sub_rg).transform(df_sub_rg)
df_rf_fitted.show(5, False)


+-----+--------------------------+-------------------------------+--------------------+----------------------+------------------------+-------------+--------------------+--------------------+-----------------------------------+----------+------------+--------------------------------------------------------------------------------------------+-----+
|Score|Incident_Weather_Condition|Incident_Road_Surface_Condition|UTA_Vehicle_Behavior|Incident_Location_Type|Incident_Light_Condition|Incident_Time|Vehicle_One_Behavior|Vehicle_Two_Behavior|Observations                       |Severity  |ARC_Decision|features                                                                                    |label|
+-----+--------------------------+-------------------------------+--------------------+----------------------+------------------------+-------------+--------------------+--------------------+-----------------------------------+----------+------------+-----------------------------------------------

In [105]:
train, test = df_rf_fitted.select('label', 'features').randomSplit([0.7, 0.3], seed=2018)

# Model Building and Evaluation

## 1. Linear Regression

In [110]:
lr = LinearRegression(labelCol='label')
lrModel = lr.fit(train)
print("Coefficients:", lrModel.coefficients)
print("Intercept:", lrModel.intercept)

Coefficients: [0.025816687920095265,0.0033531143316688035,-0.09069326223357876,-0.03897653839200875,-0.13300673819202777,-0.11820850497190633,-0.49654360641854745,-0.3119203558098298,-0.6880893584284584,-0.08722067487319579,-0.07818324690730004,0.6427800668023337,-0.08511842711244626,-0.017962636118667587,0.07334653128070019,0.07768561528410531,0.08296797429970891,0.05730892630821374,-0.028020097234453045,3.227012329346484,-0.006119264684217613,-0.010629775516700097,-0.007733279269783499,0.0010210159403962295,-0.006677179809437616,-0.003631417364941118,0.060324661965237586,0.03129337931841181,0.06243698914796838,0.015088679220641601,0.005360725086836298,-0.011370956476643306,0.027399165470653956,-0.005606967247484565,-0.021353131558356914,-0.17704952450531844,-0.08015833025238084,-0.1019650657870589,-5.5423815949942826e-05,-0.007116842401046231,-0.034153188322056285,0.3125279046168335,0.024261853648115825,-0.0015141530946449564,-0.01780620062557604,0.021012541868065622,0.04810553325836

In [111]:
e = lrModel.evaluate(test)
import numpy as np

print("LR:","MAE:",np.round(e.meanAbsoluteError,2),"RMSE:",np.round(e.rootMeanSquaredError,2),"R2:",np.round(e.r2,2))

LR: MAE: 0.2 RMSE: 3.22 R2: 1.0


In [112]:
import numpy as np
def RegressionModelsEvaluator(name, predictions):    
    predictionAndLabels =predictions.select("prediction", "label").rdd
    metrics = RegressionMetrics(predictionAndLabels)
    print(name,"MAE:", np.round(metrics.meanAbsoluteError,2),"RMSE:", np.round(metrics.rootMeanSquaredError,2),"R2:", np.round(metrics.r2,2))

## 2. Generalized Linear Regression

In [115]:
glr = GeneralizedLinearRegression(labelCol='label')
glrModel = glr.fit(train)

#print("Coefficients:",glrModel.coefficients)
#print("Intercept:",glrModel.intercept)
#print("p-values", glrModel.summary.pValues)
print("")

predictions = glrModel.transform(test)
RegressionModelsEvaluator("GLR:",predictions)


GLR: MAE: 0.2 RMSE: 3.22 R2: 1.0


## 3. Decision Tree Regression

In [118]:
dtr = DecisionTreeRegressor(labelCol='label', maxDepth=4)
dtrModel = dtr.fit(train)

predictions = dtrModel.transform(test)
RegressionModelsEvaluator("DT with Depth 4:",predictions)

print("")


dtr2 = DecisionTreeRegressor(labelCol='label')
predictions = dtr2.fit(train).transform(test)
RegressionModelsEvaluator("DT:",predictions)


DT with Depth 4: MAE: 4.67 RMSE: 17.23 R2: 0.96

DT: MAE: 3.35 RMSE: 16.4 R2: 0.96


## 4. Random Forest Regression

In [120]:
rfr = RandomForestRegressor(labelCol='label', numTrees=3, maxDepth=3)
rfrModel = rfr.fit(train)

predictions = rfrModel.transform(test)
RegressionModelsEvaluator("RF with numTress:3 Depth:3 :",predictions)

print("")


rfr2 = RandomForestRegressor(labelCol='label')
predictions = rfr2.fit(train).transform(test)
RegressionModelsEvaluator("RF2:",predictions)

RF with numTress:3 Depth:3 : MAE: 8.58 RMSE: 25.96 R2: 0.91

RF2: MAE: 4.83 RMSE: 18.58 R2: 0.95


## 5. Gradient - Boosted Tree Regression

In [121]:
gbtr = GBTRegressor(labelCol='label', maxDepth=3, maxIter=3)
gbtrModel = gbtr.fit(train)


predictions = gbtrModel.transform(test)
RegressionModelsEvaluator("GB with Depth:3 MaxItr:3 :",predictions)

print("")


gbtr2 = GBTRegressor(labelCol='label')
predictions = gbtr2.fit(train).transform(test)
RegressionModelsEvaluator("GB2:",predictions)

GB with Depth:3 MaxItr:3 : MAE: 5.44 RMSE: 19.06 R2: 0.95

GB2: MAE: 2.31 RMSE: 11.61 R2: 0.98


## Parameter Grid tuning for trees : GBTR, RFR and DTR

In [123]:
# ParamGrid_gbtr
paramGrid_gbtr = (ParamGridBuilder()
             .addGrid(gbtr.maxDepth, [2,4,6,8])
             .addGrid(gbtr.maxIter, [2,4,6,8])
             .build())

evaluator = RegressionEvaluator(labelCol='label', metricName='rmse')
gbtrTuned = TrainValidationSplit(estimator=gbtr2, estimatorParamMaps=paramGrid_gbtr, evaluator=evaluator, trainRatio=0.6)
gbtrTunedModel = gbtrTuned.fit(train)

split_predictions = gbtrTunedModel.transform(test)
RegressionModelsEvaluator("gbtrTunedModel:",split_predictions)


gbtrTunedModel: MAE: 2.31 RMSE: 11.61 R2: 0.98


In [124]:
# ParamGrid_rfr
paramGrid_rfr = (ParamGridBuilder()
             .addGrid(rfr.maxDepth, [2,4,6,8,10])
             .addGrid(rfr.numTrees, [50,100,150])
             .build())

evaluator = RegressionEvaluator(labelCol='label', metricName='rmse')
rfrTuned = TrainValidationSplit(estimator=rfr2, estimatorParamMaps=paramGrid_rfr, evaluator=evaluator, trainRatio=0.6)
rfrTunedModel = rfrTuned.fit(train)

split_predictions = rfrTunedModel.transform(test)
RegressionModelsEvaluator("rfrTunedModel:",split_predictions)

rfrTunedModel: MAE: 4.83 RMSE: 18.58 R2: 0.95


In [126]:
# ParamGrid_dtr
paramGrid_dtr = (ParamGridBuilder()
             .addGrid(dtr.maxDepth, [2,3,4,5])
             .build())


evaluator = RegressionEvaluator(labelCol='label', metricName='rmse')
dtrTuned = TrainValidationSplit(estimator=dtr2, estimatorParamMaps=paramGrid_dtr, evaluator=evaluator, trainRatio=0.6)
dtrTunedModel = dtrTuned.fit(train)

split_predictions = dtrTunedModel.transform(test)
RegressionModelsEvaluator("dtrTunedModel:",split_predictions)

dtrTunedModel: MAE: 3.35 RMSE: 16.4 R2: 0.96


On comparing the evaluation results we found that Linear Regression and Generalized Linear Regression are performing better than the other regression models.